# Tutorial: create a time series stack from OPERA DISP-S1 data

## 0. Environment Setup

This will require an Earthdata username/password, either as a `~/.netrc` file:

First install `opera-utils` with the optional disp dependency (or via conda-forge):

In [ ]:
HAS_LATEST_OPERA_UTILS = False
try:
    import opera_utils

    HAS_LATEST_OPERA_UTILS = opera_utils.__version__ >= "0.23.3"
except ImportError:
    pass
if not HAS_LATEST_OPERA_UTILS:
    !pip install "opera-utils[disp]>=0.23.3"

Next, make sure we have our ~/.netrc file set up with earthdata credentials.
If you don't have a profile, you can [register here](https://urs.earthdata.nasa.gov/users/new).

```
$ cat ~/.netrc
machine urs.earthdata.nasa.gov
        login your_username
        password your_password
```



In [ ]:
!pip install tinynetrc


In [ ]:

from pathlib import Path

from tinynetrc import Netrc

EARTHDATA_USERNAME = "myusername"
EARTHDATA_PASSWORD = "CHANGEME"
p = Path.home() / ".netrc"
p.touch(mode=0o600, exist_ok=True)
with Netrc(p) as n:
    n["urs.earthdata.nasa.gov"] = {
        "login": EARTHDATA_USERNAME,
        "password": EARTHDATA_PASSWORD,
    }


In [ ]:

import opera_utils.credentials

try:
    opera_utils.credentials.get_earthdata_username_password(
        earthdata_username=None, earthdata_password=None
    )
except opera_utils.credentials.EarthdataLoginError:
    print("Please set up your earthdata credentials")
    raise

## 1. Download a Frame Subset Over West Texas

Now we'll run the download command to get a geographic subset of the DISP-S1 NetCDF files.


Other options include:

- `--wkt`: define a polygon to download based on a well-known text string
- `--url-type S3`: download directly from S3 (if you are running this on an EC2 instance in the "us-west-2" region)
- `--start-datetime` and `--end-datetime`: define a time range to download

In [ ]:
%%bash
opera-utils disp-s1-download \
    --bbox -102.71 31.35 -102.6 31.45 \
    --frame-id 20697 \
    --output-dir subsets-west-texas \
    --start-datetime 2021-01-01 \
    --end-datetime 2024-01-01 \
    --num-workers 4


## 2. Reformat the data to remove the "moving reference"

We'll create one 3D stack of data with all the layers in it that has taken care of the moving reference date.
This lets us plot the continuous time series from the start without jumps in the time series.
We're going to use the "border" referencing, since we're downloading a small land area surrounding one uplift.
This will spatially reference each time series epoch to the average of the border pixels.

To see all options for setting up this stack, run `opera-utils disp-s1-reformat --help`

In [ ]:
%%bash
opera-utils disp-s1-reformat \
    --output-name stack-west-texas.zarr \
    --input-files subsets-west-texas/OPERA_L3_DISP-S1*.nc \
    --reference-method BORDER

## 3. Plot the results



In [ ]:
try:
    import matplotlib.pyplot as plt
except ImportError as e:
    print(f"Failed to import matplotlib: {e}")
    !pip install matplotlib
import numpy as np
import xarray as xr

%matplotlib inline

In [ ]:
ds = xr.open_dataset("stack-west-texas.zarr", consolidated=False)
ds

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(10, 4))

# Plot the cumulative displacement
last_displacement = ds.displacement.isel(time=-1)
last_displacement.plot.imshow(ax=axes[0])

# Plot the time series at the maximum uplift
row, col = np.unravel_index(
    np.nanargmax(last_displacement.values), last_displacement.shape
)
ds.displacement.isel(y=row, x=col).plot(ax=axes[1], marker=".")
fig.tight_layout()